<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/Lecture-AI_programming/week09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week09
## 모델 저장하기

In [1]:
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

## `state_dict()`
`torch.nn.Module` 객체의 `state_dict()` 메서드.

- 모델의 학습 가능한 parameters의 상태와
- 버퍼(예: BatchNorm의 running mean과 variance 등)의 상태를 저장하는
- collections.OrderedDict 객체를 반환.
- 반환된 객체는 모델의 현재 상태를 나타내며, 저장 및 로드가 가능.

1. OrderedDict 형태:
- attribute 이름을 키로 하고,
- 그에 대응하는 torch.Tensor를 값으로 갖는
- `collections.OrderedDict` 객체를 반환.
2. 학습 가능한 매개변수:
- `torch.nn.Parameter` 객체로 정의된 모든 학습 가능한 attributes를 포함.
3. 버퍼:
- 모델에 포함된 모든 버퍼(예: BatchNorm 계층의 running mean과 variance)도 포함.

- Serialization: 개체를 key: value 형태로 하여 직렬화로 저장.
- OrderedDict: 순서가 정의된 딕셔너리 형태.



## parameters
`state_dict(desitnation=None, prefix='', keep_vars=False)`
### keep_vars
buffers와 parameters의 값만을 추출할지 결정
- 저장의 용도로는 keep_vars=False를 사용하는 게 좋음.

**keep_vars=True인 경우**
- value가 parameter인 경우엔 Parameter 로 얻어짐.
- value가 buffer인 경우엔 Tensor 로 얻어짐.
- 메모리 사용량이 커지고, 매우 느리고 복잡한 동작 발생.

**장점**
- 모델 디버깅: 모델 상태를 조사하고 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
- 모델 커스터마이징: 모델을 불러온 후 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
- 모델 저장 및 불러오기 확장: 모델 저장 및 불러오기 프로세스를 확장하고 추가적인 정보를 저장해야 하는 경우 유용



In [ ]:
# 간단한 모델 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear = nn.Linear(10, 5)
        self.bn = nn.BatchNorm1d(5)

    def forward(self, x):
        x = self.linear(x)
        x = self.bn(x)
        return x

# 모델 인스턴스 생성
model = MyModel()

# 모델의 state_dict 가져오기
state_dict = model.state_dict()

# state_dict 내용 출력
for key, value in state_dict.items():
    print(f"{key}: {value.shape}")

linear.weight: torch.Size([5, 10])
linear.bias: torch.Size([5])
bn.weight: torch.Size([5])
bn.bias: torch.Size([5])
bn.running_mean: torch.Size([5])
bn.running_var: torch.Size([5])
bn.num_batches_tracked: torch.Size([])


### `linear`: `nn.Linear`
- `linear.weight`: weight parameter
- `linear.bias`: bias parameter

### `bn`: `nn.BatchNorm1d`
- `bn.weight`: weight parameter
- `bn.bias`: bias parameter
- `bn.running_mean`: running mean buffer
- `bn.running_var`: running variance buffer
- `bn.num_batches_tracked`: 배치의 수 추적 buffer

In [ ]:
# 모델의 state_dict 가져오기
state_dict0 = model.state_dict(prefix='ds', keep_vars=True)

# state_dict 내용 출력
for key, value in state_dict0.items():
    print(f"{key}: {value.shape}")

dslinear.weight: torch.Size([5, 10])
dslinear.bias: torch.Size([5])
dsbn.weight: torch.Size([5])
dsbn.bias: torch.Size([5])
dsbn.running_mean: torch.Size([5])
dsbn.running_var: torch.Size([5])
dsbn.num_batches_tracked: torch.Size([])


## 모델 저장

state_dict를 파일에 저장하여 나중에 모델을 복원할 수 있음.

In [ ]:
torch.save(model.state_dict(), 'model_state.pth')

## 모델 로드

저장된 state_dict를 로드하여 모델을 복원.

In [ ]:
model = MyModel()
model.load_state_dict(torch.load('model_state.pth'))

<All keys matched successfully>

### 파라미터 업데이트

state_dict를 사용하여 모델의 특정 파라미터를 업데이트할 수 있음.

In [ ]:
state_dict['linear.weight'] = torch.ones_like(state_dict['linear.weight'])
model.load_state_dict(state_dict)

<All keys matched successfully>

<hr>

# Save and Load Model


## 모델의 Parameters 저장
- Structure 등은 저장되지 않음.
- PyTorch 버전에 관계없이 사용 가능.

### `state_dict()`
- 객체의 상태를 가지고 있어 상태를 저장하고 복원할 수 있음.
- 모델의 parameter 상태와 buffer의 상태 저장
- 현재 Module instance의 상태에 해당하는 OrderedDict 객체 `state_dict` 반환.


### `load_state_dict(state_dict)`
현재 Module 인스턴스의 상태를 argument로 넘겨진 OrderedDict 객체 state_dict를 이용해 설정

- missing_keys : 로드하려는 state_dict에는 있으나 load_state_dict메서드를 호출한 Module 객체에는 없는 키들.
- unexpected_keys : Module 객체에는 있으나 인자로 넘겨진 state_dict에는 없는 키들.

### `torch.save(state_dict,'file_path') & state_dict=torch.load('file_path')`

`state_dict`객체는 torch의 `save` `load`를 이용해 파일로 저장되거나 로딩됨.

In [ ]:
class Model(nn.Module):
  def __init__(self, n_in_f, n_out_f, w, b):
    super().__init__()
    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    init.constant_(self.l0.weight, w)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, b)

  def forward(self, x):
    return self.l0(x)

In [ ]:
# 모델 객체를 생성하고, 이에 대한 파라메터 확인후
# 파라메터만 저장.
model1 = Model(3, 1, 1., 0.5)
print(list(model1.named_parameters()))
torch.save(model1.state_dict(), 'model_params.pth')

# 새로운 모델 객체를 생성.
# 해당 모델 객체는 구조는 같으나, 파라메터들의 초기값은 다름.
model2 = Model(3, 1, 2., 1.5)

print('===============')
for old, new in zip(model1.parameters(), model2.parameters()):

  if not torch.equal(old,new):
    print('model and n_model w/ default init do not have parameters with the same values!')
    break
else:
  print('model and n_model w/ default init have parameters with the same values!')
print('===============')

# 이전 저장한 parameters에 대한 state_dict를
# 로드하고 해당 state_dict로 새로만든 모델의
# 파라메터를 설정하고 이전 모델과 비교.
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')
print(f'{type(loaded_params_ordered_dict)=}') # collections.OredredDict

ret_v = model2.load_state_dict(loaded_params_ordered_dict)
print(f'{type(ret_v)}: {ret_v}')

print('===============')
for old, new in zip(model1.parameters(), model2.parameters()):

  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]
model and n_model w/ default init do not have parameters with the same values!
type(loaded_params_ordered_dict)=<class 'collections.OrderedDict'>
<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
model and n_model have parameters with the same values!



## model 전체 저장
- Parameters와 Structure 함께 저장
- pickle에 의존하여 인스턴스를 직렬화하므로 모델 클래스 정의가 필요함.
- 버전이 크게 바뀔 경우 안 될 확률이 매우 높으므로 주기적으로 최신 버전으로 다시 저장 필요.

In [ ]:
# 저장할 모델 생성.
model = Model(3, 1, 2., 1.5)

# 모델 저장
torch.save(model, 'model.pth')

# 저장된 model 로드.
n_model = torch.load('model.pth')
print(f'{type(n_model)=}, {n_model}')

# 두 모델의 parameters비교.
for old, new in zip(model.parameters(), n_model.parameters()):
  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

type(n_model)=<class '__main__.Model'>, Model(
  (l0): Linear(in_features=3, out_features=1, bias=True)
)
model and n_model have parameters with the same values!


<hr>

# Tensor 비교

PyTorch에서 `nn.Parameter` 또는 `tensor` 객체 두 개가 같은 값을 가지는지 확인하는 방법.
- `torch.equal` : 두 텐서의 모든 요소가 동일한지 확인
- `torch.allclose` : 지정된 허용 오차 내에서 두 텐서가 거의 동일한지 확인.

## `torch.equal` 사용
- 완벽히 같은 값을 가질 때만 `True` 반환.

In [2]:
# nn.Parameter 객체 생성
param1 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param3 = nn.Parameter(torch.tensor([1.0, 2.0, 4.0]))

# 값 비교
print(torch.equal(param1, param2))  # True
print(torch.equal(param1, param3))  # False

True
False


## `torch.allclose` 사용
- 두 텐서가 지정된 허용 오차 내에서 거의 동일한지 확인.
- 부동 소수점 연산의 미세한 차이로 인한 불일치를 허용할 수 있음

In [8]:
param2 = nn.Parameter(torch.tensor([1.0, 2.00001, 3.0]))

# 값 비교
print(torch.allclose(param1, param2))  # True, 기본 허용 오차 내
print(torch.allclose(param1, param3))  # False

True
False


### 허용 오차 설정
`torch.allclose(tensor1, tensor2, rtol=1e-05, atol=1e-08)`
- 절대적(atol), 상대적(rtol) 허용 오차 설정 가능.

In [14]:
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0001]))

print(torch.allclose(param1, param2, atol=1e-4))  # True, 허용 오차 내
print(torch.allclose(param1, param2, atol=1e-5))  # False, 허용 오차 밖

True
False


### 절대적 오차 (`atol`)
$\mathrm{atol} = |a-b|$
- 두 값의 차이가 일정한 허용 오차인지 확인.
- 값 크기에 상관없이 일정

$ a = 100$이고 $b = 100.1$이면

$\mathrm{atol} = |100-100.1| = 0.1$


### 상대적 오차 (`rtol`)
$\mathrm{rtol} = \cfrac{|a-b|}{|b|}$
- 두 값의 차이가 값에 크기에 비례한 허용 오차인지 확인.
- 값 크기에 비례함.

$ a = 100$이고 $b = 100.1$이면

$\mathrm{rtol} = \cfrac{|100-100.1|}{|100.1|} \approx 0.001$


### `torch.allclose`의 유사 기준
$|a_i - b_i| \leq \mathrm{atol}+|b_i| \times \mathrm{rtol}$

<hr>

## `nn.Sequential`

## `nn.ModuleList`

## `nn.ModulDict`

https://ds31x.tistory.com/266

이름을 지정하면 디버깅시 확인이 매우 편함.

일반 list / dict를 사용하면 선언이 되지 않고 parameter로 선언이 안 되기 때문에 하습 자체가 불가.

loss가 기존보다 줄었다면 모델 저장 기능 추가. -> checkpoint.
epoch마다 저장하는 것은 파일명에 에포크수가 포함되어 있어야 함.
+날짜 및 시간

optimizer의 상태도 그대로 저장해야 함.